# AIM
- run more baseline models with sklearn pipeline
- input data: data with preprocessed routine as of Mar22

Created: 23 Mar 2022

# LIBRARIES

In [1]:
##### MODULES
import os
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import matplotlib.dates
import matplotlib.patches
import datetime as dt
import ast

## custom modules
# import sys  
# sys.path.append('../../scripts')

import import_data
import clean_data
import mappings
import plotting
import report
import helper

##### OPTIONS
pd.options.mode.chained_assignment = None

# autoreload external modules after saving changes to disk
%reload_ext autoreload
%autoreload 2

##### DIRECTORIES
proj_dir = Path('.') / '..' / '..'
source_data_dir = proj_dir/'data'/'source'
clean_data_dir =  proj_dir/'data'/'clean'
viz_dir = proj_dir/'viz'

In [2]:
import random
from pprint import pprint
import feather

ML libraries

In [3]:
from sklearn.model_selection import train_test_split

# preprocessing
from sklearn.preprocessing import StandardScaler


# classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import svm

# tune
from sklearn.model_selection import GridSearchCV

# performance
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# pipe
from sklearn.pipeline import Pipeline

# PARAMETERS

In [4]:
PATH_TO_PREPROCESSED_DATA = proj_dir/'data'/'clean'/'preprocessMar22'/'data_merge2.ftr'
PATH_TO_FEATURES_SELECTED = proj_dir/'data'/'clean'/'fsMar23all'/'feat_tsfresh_select.ftr'

SEED = 123

### INITIAL SPLIT
TEST_SIZE = 0.3

### TUNING
N_JOBS = -2 # all but 1 CPUs
KFOLD = 5
SCORING = 'balanced_accuracy'

### CROSS VALIDATION
N_SPLITS = 5
N_REPEATS = 3

In [5]:
tune_defaults = {
    "n_jobs": N_JOBS,
    "cv": KFOLD,
    "scoring": SCORING,
    "return_train_score": True
}


# IMPORT

In [6]:
fs = pd.read_feather(PATH_TO_FEATURES_SELECTED)
df = pd.read_feather(PATH_TO_PREPROCESSED_DATA)

In [7]:
df['target'] = df['phq'] > 10
_, y = clean_data.generate_ts_y(df)
y = y[fs['index']]

# check index does match
if np.array_equal(y.index.values, fs['index'].values):
    print("Primary key does match.")

Primary key does match.


In [8]:
X = fs.loc[:, fs.columns != 'index'].copy()

# TRAIN TEST SPLIT

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED, test_size=TEST_SIZE)

report.report_train_test_split(X_train, X_test, y_train, y_test)

Training Features Shape: (2002, 422)
Training Labels Shape: (2002,)
Testing Features Shape: (858, 422)
Testing Labels Shape: (858,)


# PIPELINE

## Tuning routine

In [10]:
def tune_hyperparameters(pipeline, params, tune_defaults=False):
    
    grid_search = GridSearchCV(pipeline, params)
    if tune_defaults:
        grid_search.set_params(**tune_defaults)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(params)

    ### fit
    grid_search.fit(X_train, y_train)
    grid_search.cv_results_.keys()

    return grid_search

KNN - standard scaling

In [11]:
params_knn = dict()
params_knn['knn__n_neighbors']=(3, 5, 7, 9, 11, 13, 15)

pipeline_knn_std = Pipeline(
    [
        ('scaler', StandardScaler()), 
        ('knn', KNeighborsClassifier())   
    ]
)

with helper.Timer("Tune: KNN"):
    search_knn_std = tune_hyperparameters(pipeline_knn_std, params_knn, tune_defaults=tune_defaults)
search_knn_std.best_params_

# [Tune: KNN]
Performing grid search...
pipeline: ['scaler', 'knn']
parameters:
{'knn__n_neighbors': (3, 5, 7, 9, 11, 13, 15)}
Elapsed: 9.7 seconds


{'knn__n_neighbors': 9}

SVC - standard scaling

In [12]:
# params_svc = dict()
# params_svc['svc__C'] = (0.1, 1, 10, 100, 1000)
# params_svc['svc__gamma'] = (1, 0.1, 0.01, 0.001, 0.0001)
# params_svc['svc__kernel'] = ['rbf']

params_svc = dict()
params_svc['svc__C'] = (1, 100)
params_svc['svc__gamma'] = (0.1, 0.001)
params_svc['svc__kernel'] = ['rbf']


pipeline_svc_std = Pipeline(
    [
        ('scaler', StandardScaler()), 
        ('svc', svm.SVC())   
    ]
)
with helper.Timer("Tune: SVC"):
    search_svc_std = tune_hyperparameters(pipeline_svc_std, params_svc, tune_defaults=tune_defaults)
search_svc_std.best_params_


# [Tune: SVC]
Performing grid search...
pipeline: ['scaler', 'svc']
parameters:
{'svc__C': (1, 100), 'svc__gamma': (0.1, 0.001), 'svc__kernel': ['rbf']}
Elapsed: 28.6 seconds


{'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}

RF - no scaling

In [13]:
# params_rf = dict()
# params_rf['rf__n_estimators'] = (10, 20, 50, 100, 200)
# params_rf['rf__min_samples_split'] = (2, 5)
# params_rf['rf__max_features'] = ["auto", "sqrt"]
# params_rf['rf__max_depth'] = [int(x) for x in np.linspace(10, 110, num = 11)] + [None]
# params_rf['rf__min_samples_leaf'] = (1,2,4)

params_rf = dict()
params_rf['rf__n_estimators'] = (10, 100)
params_rf['rf__min_samples_split'] = (2, 5)
params_rf['rf__max_features'] = ["auto", "sqrt"]
params_rf['rf__max_depth'] = [int(x) for x in np.linspace(10, 110, num = 2)] + [None]
params_rf['rf__min_samples_leaf'] = (1,4)

pipeline_rf = Pipeline(
    [ 
        ('rf', RandomForestClassifier(random_state=SEED))   
    ]
)
with helper.Timer("Tune: RF"):
    search_rf = tune_hyperparameters(pipeline_rf, params_rf, tune_defaults)
search_rf.best_params_

# [Tune: RF]
Performing grid search...
pipeline: ['rf']
parameters:
{'rf__max_depth': [10, 110, None],
 'rf__max_features': ['auto', 'sqrt'],
 'rf__min_samples_leaf': (1, 4),
 'rf__min_samples_split': (2, 5),
 'rf__n_estimators': (10, 100)}
Elapsed: 84.9 seconds


{'rf__max_depth': 10,
 'rf__max_features': 'auto',
 'rf__min_samples_leaf': 4,
 'rf__min_samples_split': 2,
 'rf__n_estimators': 100}

Logistic regression, regularized

In [14]:
from sklearn.linear_model import LogisticRegression

#! add prefix
# params_lr = dict()
# params_lr['logistic__solver'] = ['newton-cg', 'lbfgs', 'liblinear']
# params_lr['logistic__penalty'] = ['none', 'l1', 'l2', 'elasticnet']
# params_lr['logistic__C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]

params_lr = dict()
params_lr['logistic__solver'] = ['newton-cg', 'lbfgs', 'liblinear']
params_lr['logistic__penalty'] = ['none', 'elasticnet']
params_lr['logistic__C'] = [1e-5, 100]


pipeline_lr = Pipeline(
    [ 
        ('logistic', LogisticRegression())   
    ]
)
with helper.Timer("Tune: SVC"):
    search_lr = tune_hyperparameters(pipeline_lr, params_lr, tune_defaults)
    
search_lr.best_params_

# [Tune: SVC]
Performing grid search...
pipeline: ['logistic']
parameters:
{'logistic__C': [1e-05, 100],
 'logistic__penalty': ['none', 'elasticnet'],
 'logistic__solver': ['newton-cg', 'lbfgs', 'liblinear']}


C:\Users\Harris\anaconda3\envs\msc-thesis\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Harris\anaconda3\envs\msc-thesis\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Harris\anaconda3\envs\msc-thesis\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Harris\anaconda3\envs\msc-thesis\lib\site-packages\sklearn\linear_model\_logistic.py", line

Elapsed: 78.1 seconds


C:\Users\Harris\anaconda3\envs\msc-thesis\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


{'logistic__C': 1e-05,
 'logistic__penalty': 'none',
 'logistic__solver': 'newton-cg'}

# Visualize tuning results

In [52]:
from sklearn.metrics import balanced_accuracy_score

searches = [search_knn_std, search_svc_std, search_rf, search_lr]
names = ['knn', 'svc', 'rf', 'logistic']

for name, search in zip(names, searches):
    print(f"### {name}")
    print(f"Best score: {search.best_score_}")
    print(f'Best params: {search.best_params_}')
    if len(search.best_params_)>1:
        pass
        # plotting.plot_search_results(search)



### knn
Best score: 0.6361371153306452
Best params: {'knn__n_neighbors': 9}
### svc
Best score: 0.6386122836211918
Best params: {'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
### rf
Best score: 0.636001979992514
Best params: {'rf__max_depth': 10, 'rf__max_features': 'auto', 'rf__min_samples_leaf': 4, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
### logistic
Best score: 0.6159062635909771
Best params: {'logistic__C': 1e-05, 'logistic__penalty': 'none', 'logistic__solver': 'newton-cg'}


# Model fitting

In [91]:
y_preds = []

for name, search in zip(names, searches):
    print(f"### {name}")

    # fit
    pipeline = search.best_estimator_
    y_pred = pipeline.predict(X_test)
    y_preds.append(y_pred) 
    
dicts = [classification_report(y_test, y_pred, output_dict=True) for y_pred in y_preds]
df = report.combine_classification_reports(dicts, names)
df.to_excel(proj_dir/"data"/"artefacts"/"baseline_models.xlsx")
df

### knn
### svc
### rf
### logistic


,name,index,precision,recall,f1-score,support
0,knn,False,0.712707,0.807933,0.757339,479.0
1,knn,True,0.707937,0.588391,0.642651,379.0
2,knn,accuracy,NaN,NaN,0.710956,858.0
3,knn,macro avg,0.710322,0.698162,0.699995,858.0
4,knn,weighted avg,0.710600,0.710956,0.706678,858.0
0,svc,False,0.658031,0.795407,0.720227,479.0
1,svc,True,0.648746,0.477573,0.550152,379.0
2,svc,accuracy,NaN,NaN,0.655012,858.0
3,svc,macro avg,0.653388,0.636490,0.635189,858.0
4,svc,weighted avg,0.653929,0.655012,0.645101,858.0
